# Oct 21, 2024: transform N162 template to the Allen CCFv3 template

In [1]:
import csv 
import os
import numpy as np
import pandas as pd
import ants
import seaborn as sns
import dill as pickle 
import shutil
import glob

from sklearn.cluster import KMeans
from tqdm import tqdm
from copy import deepcopy

# ignore user warnings
import warnings
warnings.filterwarnings("ignore") #, category=UserWarning)

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

In [3]:
BASE_path = f'{os.environ["HOME"]}/mouse_dataset'
os.system(f'mkdir -p {BASE_path}/yale_to_orig_template')

0

In [4]:
yale_template_lpi_file = f'/home/govindas/new_mouse_dataset/data/Symmetric_N162_0.20.nii.gz'
yale_template_lpi_img = ants.image_read(yale_template_lpi_file)
yale_template_lpi = yale_template_lpi_img.numpy()
yale_template_lpi_img

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (60, 46, 81)
	 Spacing    : (0.2, 0.2, 0.2)
	 Origin     : (18.2, 2.8, -7.8)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [5]:
yale_template_ras_file = f'/home/govindas/new_mouse_dataset/data/Symmetric_N162_0.20_RAS.nii.gz'
yale_template_ras_img = ants.image_read(yale_template_ras_file)
yale_template_ras = yale_template_ras_img.numpy()
yale_template_ras_img

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (60, 81, 46)
	 Spacing    : (0.2, 0.2, 0.2)
	 Origin     : (18.2, 2.8, -7.8)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [6]:
yale_template_file = f'{BASE_path}/gabe_resampled_DSURQE/DSURQE_average_to_N162_RAS.nii.gz'
yale_template_img = ants.image_read(yale_template_file)
yale_template = yale_template_img.numpy()

yale_mask_file = f'{BASE_path}/gabe_resampled_DSURQE/DSURQE_mask_to_N162_RAS.nii.gz'
yale_mask_img = ants.image_read(yale_mask_file)
yale_mask = yale_mask_img.numpy()

yale_template *= yale_mask
yale_template_img = yale_template_img.new_image_like(yale_template)
yale_template_img.to_file(f'{BASE_path}/gabe_resampled_DSURQE/DSURQE_average_to_N162_RAS_masked.nii.gz')

display(yale_template_img), display(yale_mask_img)

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (118, 160, 90)
	 Spacing    : (0.1, 0.1, 0.1)
	 Origin     : (18.1, 2.7, -7.8)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (118, 160, 90)
	 Spacing    : (0.1, 0.1, 0.1)
	 Origin     : (18.1, 2.7, -7.8)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

(None, None)

In [26]:
orig_template_file = f'{BASE_path}/mouse-brain-templates-0.5.3/dsurqec_200micron_masked.nii'
orig_template_img = ants.image_read(orig_template_file)
orig_template = orig_template_img.numpy()

orig_template_img.to_file(f'{BASE_path}/yale_to_orig_template/original_template.nii.gz')

display(orig_template_img)

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (63, 96, 48)
	 Spacing    : (0.2, 0.2, 0.2)
	 Origin     : (6.27, 10.6, -7.88)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [8]:
# resample dsurqe template to 200um
os.system('export ITK_GLOBAL_DEFAULT_NUMBER_OF_THREADS=1')
os.system('export ANTS_RANDOM_SEED=1')

yale_template_rs_img = ants.resample_image_to_target(
    image=yale_template_img,
    target=yale_template_ras_img,
)
yale_template_rs_img

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (60, 81, 46)
	 Spacing    : (0.2, 0.2, 0.2)
	 Origin     : (18.2, 2.8, -7.8)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [9]:
# linear registration
os.system('export ITK_GLOBAL_DEFAULT_NUMBER_OF_THREADS=1')
os.system('export ANTS_RANDOM_SEED=1')

tx_trsaa = ants.registration(
    fixed=orig_template_img,
    moving=yale_template_rs_img, 
    type_of_transform='TRSAA',
    random_seed=args.SEED,
)

In [10]:
yale_template_tx_img = ants.apply_transforms(
    fixed=orig_template_img,
    moving=yale_template_rs_img,
    transformlist=tx_trsaa['fwdtransforms'],
    interpolator='linear',
)
yale_template_tx_img

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (63, 96, 48)
	 Spacing    : (0.2, 0.2, 0.2)
	 Origin     : (6.27, 10.6, -7.88)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [11]:
yale_template_tx_img.to_file(f'{BASE_path}/yale_to_orig_template/yale_template_tx_lin.nii.gz')

In [12]:
# non linear registration
os.system('export ITK_GLOBAL_DEFAULT_NUMBER_OF_THREADS=1')
os.system('export ANTS_RANDOM_SEED=1')

tx_syn = ants.registration(
    fixed=orig_template_img,
    moving=yale_template_tx_img,
    type_of_transform='SyN',
    random_seed=args.SEED,
)

In [13]:
yale_template_tx_img = ants.apply_transforms(
    fixed=orig_template_img,
    moving=yale_template_tx_img,
    transformlist=tx_syn['fwdtransforms'],
    interpolator='linear',
)
yale_template_tx_img

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (63, 96, 48)
	 Spacing    : (0.2, 0.2, 0.2)
	 Origin     : (6.27, 10.6, -7.88)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [14]:
yale_template_tx_img.to_file(f'{BASE_path}/yale_to_orig_template/yale_template_tx_nonlin.nii.gz')

In [15]:
yale_mask_img = yale_template_tx_img.get_mask()
yale_mask_img.to_file(f'{BASE_path}/yale_to_orig_template/yale_template_tx_nonlin_mask.nii.gz')
yale_mask = yale_mask_img.numpy()
np.min(yale_mask), np.max(yale_mask)

(0.0, 1.0)

In [16]:
orig_mask_img = orig_template_img.get_mask()
orig_mask_img.to_file(f'{BASE_path}/yale_to_orig_template/orig_template_mask.nii.gz')

In [17]:
# save transforms
import shutil
for idx, tx in enumerate(tx_trsaa['fwdtransforms']):
    endswith = '.'.join(tx.split('.')[1:])
    shutil.copy2(tx, f'{BASE_path}/yale_to_orig_template/transformation_lin{idx:02d}.{endswith}')

for idx, tx in enumerate(tx_syn['fwdtransforms']):
    endswith = '.'.join(tx.split('.')[1:])
    shutil.copy2(tx, f'{BASE_path}/yale_to_orig_template/transformation_nonlin{idx:02d}.{endswith}')

In [18]:
txs_list = sorted(glob.glob(f'{BASE_path}/yale_to_orig_template/transformation*', recursive=True))

In [19]:
# transform image to the original space

In [20]:
func_img = yale_template_lpi_img

In [24]:
# reorient to ras orientation
func = func_img.numpy()
func = func.transpose(0, 2, 1)
func_ro_img = yale_template_ras_img.new_image_like(func)
func_ro_img

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (60, 81, 46)
	 Spacing    : (0.2, 0.2, 0.2)
	 Origin     : (18.2, 2.8, -7.8)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [25]:
func_ro_tx_img = ants.apply_transforms(
    fixed=orig_template_img,
    moving=func_ro_img,
    transformlist=txs_list,
    interpolator='linear',
)
func_ro_tx_img.to_file(f'{BASE_path}/yale_to_orig_template/yale_template_tx_test.nii.gz')
func_ro_tx_img

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (63, 96, 48)
	 Spacing    : (0.2, 0.2, 0.2)
	 Origin     : (6.27, 10.6, -7.88)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]